In [23]:
import os
import glob
import numpy as np
import pandas as pd
import pickle
import datetime
from datetime import timedelta

In [24]:
src_dir = './meteo_data/'
meteo_dir = src_dir + 'Merged/'
output_dir = './output/'

In [25]:
not_exist_site = src_dir + 'samples_out_of_range.csv'
site_mapping = src_dir + 'mapping.csv'
unique_site = src_dir + 'uniqueSite.csv'

In [26]:
not_exist_data = pd.read_csv(not_exist_site)
not_exist_list = not_exist_data['Site_ID'].to_list()

site_mapping_data = pd.read_csv(site_mapping)
#site_mapping_data

unique_site_data = pd.read_csv(unique_site)
unique_site_list = unique_site_data['Site_ID'].to_list()

site_infos = pd.read_csv('AmeriFlux_siteinfo.csv')

In [27]:
meteo_files = glob.glob(meteo_dir+'*.csv')

In [28]:
ameri_sites_data = pd.read_csv('AmeriFlux_siteinfo.csv')
ameri_sites_list = ameri_sites_data['site_id'].to_list()
ameri_sites_list

['AR-TF1',
 'AR-TF2',
 'BR-CST',
 'BR-Npw',
 'CA-ARB',
 'CA-ARF',
 'CA-Ca1',
 'CA-Ca2',
 'CA-Ca3',
 'CA-Cbo',
 'CA-Cha',
 'CA-DB2',
 'CA-DBB',
 'CA-ER1',
 'CA-LP1',
 'CA-MA1',
 'CA-MA2',
 'CA-MA3',
 'CA-Man',
 'CA-Na1',
 'CA-Oas',
 'CA-Obs',
 'CA-SF1',
 'CA-SF2',
 'CA-SF3',
 'CA-TP1',
 'CA-TP2',
 'CA-TP3',
 'CA-TP4',
 'CA-TPD',
 'CL-SDF',
 'CL-SDP',
 'CR-SoC',
 'MX-Aog',
 'MX-PMm',
 'MX-Tes',
 'PE-QFR',
 'PR-xGU',
 'PR-xLA',
 'US-A03',
 'US-A10',
 'US-A32',
 'US-A74',
 'US-Act',
 'US-Akn',
 'US-ALQ',
 'US-An1',
 'US-An2',
 'US-An3',
 'US-AR1',
 'US-AR2',
 'US-ARM',
 'US-ASH',
 'US-ASL',
 'US-ASM',
 'US-Bar',
 'US-Bi1',
 'US-Bi2',
 'US-Blo',
 'US-BMM',
 'US-Bo1',
 'US-Bo2',
 'US-BRG',
 'US-Bsg',
 'US-BZB',
 'US-BZF',
 'US-BZo',
 'US-BZS',
 'US-CC1',
 'US-CC2',
 'US-CdM',
 'US-Ced',
 'US-CF1',
 'US-CF2',
 'US-CF3',
 'US-CF4',
 'US-CMW',
 'US-Cop',
 'US-CPk',
 'US-CRT',
 'US-CS1',
 'US-CS2',
 'US-CS3',
 'US-CS4',
 'US-CS5',
 'US-DFC',
 'US-DFK',
 'US-Dia',
 'US-Dix',
 'US-Dk1',
 'US-Dk2',

In [29]:
len(ameri_sites_list), len(meteo_files)

(333, 182)

In [30]:
#split rows per years
# return year list and start and end position for each year
# src_data is Timestamp like 2021-12-31 14:00:00
def split_records(src_data) -> (list, list, list):
    years = []
    start_pos = []
    end_pos = []

    start_year = -1

    for idx in range(src_data.size):
        if start_year == -1:
            start_year = src_data[idx].year
            years.append(start_year)
            start_pos.append(idx)
        else:
            if start_year == src_data[idx].year:
                _end = idx
            else:
                end_pos.append(_end)
                start_year = src_data[idx].year
                years.append(start_year)
                start_pos.append(idx)

    end_pos.append(_end)
    
    return years, start_pos, end_pos

In [31]:
def create_vpd(row):
    sh = row['spfh']
    ta = row['tair']
    pa = row['psurf']/1000.0

    es = 0.6108 * np.exp((17.27*ta)/(ta+237.3)) * 1000
    ea = sh * (pa*1000) / (0.378*sh + 0.622)    # [Pa]
    if ea > es:
        ea = es
    vpd = (es - ea) /100.0

    return round(vpd,4)

In [32]:
selected_cols = ['tair', 'psurf','VPD_era', 'wind','precip','swdown','lwdown']

In [33]:
new_col_names = {'tair':'Ta_era', 'psurf':'Pa_era', 'wind':'WS_era','precip':'Precip_era', 'swdown':'Rg_era',
                 'lwdown':'LWin_era'}

In [34]:
def create_timestamp(row):
    year = row['Year']
    month = row['Month']
    day = row['Day']
    hour = row['Hour']
    dt = datetime.datetime(year,month,day,hour)

    return dt + pd.Timedelta('%dhour'%utc)

In [35]:
def append_rows(df, utc):
    # use 12-31 00:00 data to fill at end
    append_dict = df.iloc[-24:-24+abs(utc)+1].to_dict('records')
    i = 0
    for row in append_dict:
        year = row['Year']
        month = row['Month']
        day = row['Day']
        hour = row['Hour']
        dt = datetime.datetime(year,month,day,hour)
        dt += timedelta(days=1)
        row['Year'] = dt.year
        row['DOY'] = i
        row['Month'] = dt.month
        row['Day'] = dt.day
        row['Hour'] = dt.hour
        i+= 1

    new_rows = pd.DataFrame(append_dict)
    
    return pd.concat([df,new_rows], axis=0,ignore_index=True)


In [ ]:
site_name

In [36]:
for site_name in ameri_sites_list:
    if site_name in not_exist_list:
        continue
        
    #if site_name in bad_sites:
    #    continue
        
    unique_id = site_mapping_data[site_mapping_data['Site_ID'] == site_name]['Mapping_to_unique'].tolist()[0]
    if unique_id not in unique_site_list:
        print("An error site {} in mapping, please check".format(unique_id))
    
    
    meteo_file = meteo_dir + unique_id + '.csv'
    if not os.path.exists(meteo_file):
        continue
        
    dst_dir = output_dir + site_name + '/06_meteo_era/'
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    
    
    print('Process site {} and file {}........'.format(site_name, meteo_file))
    df = pd.read_csv(meteo_file,dtype={'Year':int,'DOY':int,'Month':int,'Day':int,'Hour':int})
    utc = site_infos[site_infos['site_id'] == site_name]['UTC'].tolist()[0]
    utc = int(utc)
    if utc < 0:
        df = append_rows(df, utc)
    
    df['Timestamp'] = df.apply(lambda row: create_timestamp(row), axis=1)
    df['VPD_era'] = df.apply (lambda row: create_vpd(row) , axis=1)
    
    
    start_t = df.iloc[0]['Timestamp']
    end_t = df.iloc[-1]['Timestamp']
    T = pd.date_range(start_t, end_t,freq='30min')
    T0 = pd.DatetimeIndex(df['Timestamp'])
    
    Ta = np.full(T.size,np.nan,np.float32)
    Ta[np.in1d(T,T0)] = df['tair'][np.in1d(T0,T)]

    Pa = np.full(T.size,np.nan,np.float32)
    Pa[np.in1d(T,T0)] = df['psurf'][np.in1d(T0,T)] / 1000

    vpd = np.full(T.size,np.nan,np.float32)
    vpd[np.in1d(T,T0)] = df['VPD_era'][np.in1d(T0,T)]

    WS = np.full(T.size,np.nan,np.float32)
    WS[np.in1d(T,T0)] = df['wind'][np.in1d(T0,T)]

    # precip is hourly accumulated value, 30min should be half value
    Precip = np.full(T.size,np.nan,np.float32)
    Precip[np.in1d(T,T0)] = df['precip'][np.in1d(T0,T)] / 2.0

    Rg = np.full(T.size,np.nan,np.float32)
    Rg[np.in1d(T,T0)] = df['swdown'][np.in1d(T0,T)]

    LWin = np.full(T.size,np.nan,np.float32)
    LWin[np.in1d(T,T0)] = df['lwdown'][np.in1d(T0,T)]
    
    new_df = pd.DataFrame({'Ta_era':Ta, 'Pa_era':Pa, 'VPD_era':vpd, 'WS_era':WS, 'Precip_era':Precip, 'Rg_era':Rg, 'LWin_era':LWin, 'LWincalc_era':LWin})
    new_df.index = T
    new_df['Ta_era'] = new_df['Ta_era'].interpolate()
    new_df['Pa_era'] = new_df['Pa_era'].interpolate()
    new_df['VPD_era'] = new_df['VPD_era'].interpolate()
    new_df['WS_era'] = new_df['WS_era'].interpolate()
    new_df['Precip_era'] = new_df['Precip_era'].interpolate()
    new_df['Rg_era'] = new_df['Rg_era'].interpolate()
    new_df['LWin_era'] = new_df['LWin_era'].interpolate()
    new_df['LWincalc_era'] = new_df['LWincalc_era'].interpolate()
    
    years, start_pos, end_pos = split_records(T)

    #site_name = 'US-Ne2'
    for year, start, end in zip(years, start_pos, end_pos):
        if year == 1979:
            continue
        if year == 2022:
            continue
        if utc > 0:
            if year == 1980: # year 1980 will missing first few records
                continue

        file_name = dst_dir + site_name + '_' + str(year) + '.csv'
        _data = new_df.iloc[start: end+1]
        _data.to_csv(file_name,index=False)
        print("    Output {}".format(file_name))
        
    

Process site CA-ARB and file ./meteo_data/Merged/CA-ARB.csv........
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1980.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1981.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1982.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1983.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1984.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1985.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1986.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1987.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1988.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1989.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1990.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1991.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1992.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1993.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1994.csv
    Output ./output/CA-ARB/06_meteo_era/CA-ARB_1995.csv
    Output ./output/CA-ARB/06_meteo_

    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_1996.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_1997.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_1998.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_1999.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2000.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2001.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2002.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2003.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2004.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2005.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2006.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2007.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2008.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2009.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2010.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2011.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2012.csv
    Output ./output/CA-Cbo/06_meteo_era/CA-Cbo_2

    Output ./output/CA-DBB/06_meteo_era/CA-DBB_2014.csv
    Output ./output/CA-DBB/06_meteo_era/CA-DBB_2015.csv
    Output ./output/CA-DBB/06_meteo_era/CA-DBB_2016.csv
    Output ./output/CA-DBB/06_meteo_era/CA-DBB_2017.csv
    Output ./output/CA-DBB/06_meteo_era/CA-DBB_2018.csv
    Output ./output/CA-DBB/06_meteo_era/CA-DBB_2019.csv
    Output ./output/CA-DBB/06_meteo_era/CA-DBB_2020.csv
    Output ./output/CA-DBB/06_meteo_era/CA-DBB_2021.csv
Process site CA-ER1 and file ./meteo_data/Merged/CA-ER1.csv........
    Output ./output/CA-ER1/06_meteo_era/CA-ER1_1980.csv
    Output ./output/CA-ER1/06_meteo_era/CA-ER1_1981.csv
    Output ./output/CA-ER1/06_meteo_era/CA-ER1_1982.csv
    Output ./output/CA-ER1/06_meteo_era/CA-ER1_1983.csv
    Output ./output/CA-ER1/06_meteo_era/CA-ER1_1984.csv
    Output ./output/CA-ER1/06_meteo_era/CA-ER1_1985.csv
    Output ./output/CA-ER1/06_meteo_era/CA-ER1_1986.csv
    Output ./output/CA-ER1/06_meteo_era/CA-ER1_1987.csv
    Output ./output/CA-ER1/06_meteo_

    Output ./output/CA-MA3/06_meteo_era/CA-MA3_1988.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_1989.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_1990.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_1991.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_1992.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_1993.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_1994.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_1995.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_1996.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_1997.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_1998.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_1999.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_2000.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_2001.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_2002.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_2003.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_2004.csv
    Output ./output/CA-MA3/06_meteo_era/CA-MA3_2

    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2006.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2007.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2008.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2009.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2010.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2011.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2012.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2013.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2014.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2015.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2016.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2017.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2018.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2019.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2020.csv
    Output ./output/CA-TP2/06_meteo_era/CA-TP2_2021.csv
Process site CA-TP3 and file ./meteo_data/Merged/CA-TP3.csv........
    Output ./output/CA-TP3/06_meteo_

...

    Output ./output/US-xSC/06_meteo_era/US-xSC_2000.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2001.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2002.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2003.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2004.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2005.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2006.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2007.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2008.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2009.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2010.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2011.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2012.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2013.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2014.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2015.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2016.csv
    Output ./output/US-xSC/06_meteo_era/US-xSC_2

    Output ./output/US-xSL/06_meteo_era/US-xSL_2018.csv
    Output ./output/US-xSL/06_meteo_era/US-xSL_2019.csv
    Output ./output/US-xSL/06_meteo_era/US-xSL_2020.csv
    Output ./output/US-xSL/06_meteo_era/US-xSL_2021.csv
Process site US-xSP and file ./meteo_data/Merged/US-xSP.csv........
    Output ./output/US-xSP/06_meteo_era/US-xSP_1980.csv
    Output ./output/US-xSP/06_meteo_era/US-xSP_1981.csv
    Output ./output/US-xSP/06_meteo_era/US-xSP_1982.csv
    Output ./output/US-xSP/06_meteo_era/US-xSP_1983.csv
    Output ./output/US-xSP/06_meteo_era/US-xSP_1984.csv
    Output ./output/US-xSP/06_meteo_era/US-xSP_1985.csv
    Output ./output/US-xSP/06_meteo_era/US-xSP_1986.csv
    Output ./output/US-xSP/06_meteo_era/US-xSP_1987.csv
    Output ./output/US-xSP/06_meteo_era/US-xSP_1988.csv
    Output ./output/US-xSP/06_meteo_era/US-xSP_1989.csv
    Output ./output/US-xSP/06_meteo_era/US-xSP_1990.csv
    Output ./output/US-xSP/06_meteo_era/US-xSP_1991.csv
    Output ./output/US-xSP/06_meteo_

    Output ./output/US-xTA/06_meteo_era/US-xTA_1992.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_1993.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_1994.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_1995.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_1996.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_1997.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_1998.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_1999.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_2000.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_2001.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_2002.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_2003.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_2004.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_2005.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_2006.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_2007.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_2008.csv
    Output ./output/US-xTA/06_meteo_era/US-xTA_2

    Output ./output/US-xUK/06_meteo_era/US-xUK_2010.csv
    Output ./output/US-xUK/06_meteo_era/US-xUK_2011.csv
    Output ./output/US-xUK/06_meteo_era/US-xUK_2012.csv
    Output ./output/US-xUK/06_meteo_era/US-xUK_2013.csv
    Output ./output/US-xUK/06_meteo_era/US-xUK_2014.csv
    Output ./output/US-xUK/06_meteo_era/US-xUK_2015.csv
    Output ./output/US-xUK/06_meteo_era/US-xUK_2016.csv
    Output ./output/US-xUK/06_meteo_era/US-xUK_2017.csv
    Output ./output/US-xUK/06_meteo_era/US-xUK_2018.csv
    Output ./output/US-xUK/06_meteo_era/US-xUK_2019.csv
    Output ./output/US-xUK/06_meteo_era/US-xUK_2020.csv
    Output ./output/US-xUK/06_meteo_era/US-xUK_2021.csv
Process site US-xUN and file ./meteo_data/Merged/US-xUN.csv........
    Output ./output/US-xUN/06_meteo_era/US-xUN_1980.csv
    Output ./output/US-xUN/06_meteo_era/US-xUN_1981.csv
    Output ./output/US-xUN/06_meteo_era/US-xUN_1982.csv
    Output ./output/US-xUN/06_meteo_era/US-xUN_1983.csv
    Output ./output/US-xUN/06_meteo_

    Output ./output/US-xYE/06_meteo_era/US-xYE_1984.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1985.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1986.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1987.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1988.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1989.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1990.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1991.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1992.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1993.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1994.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1995.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1996.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1997.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1998.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_1999.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_2000.csv
    Output ./output/US-xYE/06_meteo_era/US-xYE_2

### Below code for debug purpose

In [ ]:
src_dir = './output/'
qcv_dir = '/01_qc_visual/qcv_files/'
era_dir = '/06_meteo_era/'

In [ ]:
for sub_dir_list in os.walk(src_dir):
    break
    
sub_dir_list[1]

In [ ]:
# Remove year 1979 csv files
for sub_dir in sub_dir_list[1]:
    site_name = sub_dir
    
    full_era_dir = src_dir + site_name + era_dir
    era_file_list = glob.glob(full_era_dir + '*.csv')
    
    era_1979 = full_era_dir + site_name + '_1979.csv'
    if os.path.exists(era_1979):
        os.remove(era_1979)
        print('remove ', era_1979)

In [ ]:
_cols = ['tair', 'precip','swdown', 'spfh', 'spRH', 'wind', 'lwdown', 'psurf']

In [ ]:
row = df.iloc[-5]
for col in _cols:
    if pd.isna(row[col]):
        print(col)

In [ ]:
row = df.iloc[-24:-24+5]
row

In [ ]:

year = row['Year']
month = row['Month']
day = row['Day']
hour = row['Hour']
dt = datetime.datetime(year,month,day,hour)
dt += timedelta(days=1)
dt


In [ ]:
append_df = df.iloc[-24:-24+5].copy()
append_df

In [ ]:
append_dict = df.iloc[-24:-24+5].to_dict('records')
i = 0
for row in append_dict:
    year = row['Year']
    month = row['Month']
    day = row['Day']
    hour = row['Hour']
    dt = datetime.datetime(year,month,day,hour)
    dt += timedelta(days=1)
    row['Year'] = dt.year
    row['DOY'] = i
    row['Month'] = dt.month
    row['Day'] = dt.day
    row['Hour'] = dt.hour
    i+= 1
    
new_rows = pd.DataFrame(append_dict)
new_rows

In [ ]:
new_df = pd.concat([df,new_rows], axis=0,ignore_index=True)

In [ ]:
df

In [ ]:
utc = site_infos[site_infos['site_id'] == site_name]['UTC'].tolist()[0]
utc

In [ ]:
site_mapping_data[site_mapping_data['Site_ID'] == site_name]

In [ ]:
df

In [ ]:
T0 = pd.DatetimeIndex(df['Timestamp'])
T0

In [ ]:
start_t = df.iloc[0]['Timestamp']
end_t = df.iloc[-1]['Timestamp']

In [ ]:
T = pd.date_range(start_t, end_t,freq='30min')
T

In [ ]:
Ta = np.full(T.size,np.nan,np.float32)
Ta[np.in1d(T,T0)] = df['tair'][np.in1d(T0,T)]

Pa = np.full(T.size,np.nan,np.float32)
Pa[np.in1d(T,T0)] = df['psurf'][np.in1d(T0,T)] / 1000

vpd = np.full(T.size,np.nan,np.float32)
vpd[np.in1d(T,T0)] = df['VPD_era'][np.in1d(T0,T)]

WS = np.full(T.size,np.nan,np.float32)
WS[np.in1d(T,T0)] = df['wind'][np.in1d(T0,T)]

# precip is hourly accumulated value, 30min should be half value
Precip = np.full(T.size,np.nan,np.float32)
Precip[np.in1d(T,T0)] = df['precip'][np.in1d(T0,T)] / 2.0

Rg = np.full(T.size,np.nan,np.float32)
Rg[np.in1d(T,T0)] = df['swdown'][np.in1d(T0,T)]

LWin = np.full(T.size,np.nan,np.float32)
LWin[np.in1d(T,T0)] = df['lwdown'][np.in1d(T0,T)]

In [ ]:
Precip

In [ ]:
new_df = pd.DataFrame({'Ta_era':Ta, 'Pa_era':Pa, 'VPD_era':vpd, 'WS_era':WS, 'Precip_era':Precip, 'Rg_era':Rg, 'LWin_era':LWin, 'LWincalc_era':LWin})
new_df.index = T
new_df['Ta_era'] = new_df['Ta_era'].interpolate()
new_df['Pa_era'] = new_df['Pa_era'].interpolate()
new_df['VPD_era'] = new_df['VPD_era'].interpolate()
new_df['WS_era'] = new_df['WS_era'].interpolate()
new_df['Precip_era'] = new_df['Precip_era'].interpolate()
new_df['Rg_era'] = new_df['Rg_era'].interpolate()
new_df['LWin_era'] = new_df['LWin_era'].interpolate()
new_df['LWincalc_era'] = new_df['LWincalc_era'].interpolate()

In [ ]:
_dt = new_df.index[0]
_dt.year

### Below code are for debugging

In [ ]:
Ta = np.full(T.size,np.nan,np.float32)
Ta[np.in1d(T,T0)] = df['tair'][np.in1d(T0,T)]

Pa = np.full(T.size,np.nan,np.float32)
Pa[np.in1d(T,T0)] = csv['pressure'][np.in1d(T0,T)] / 1000

SH = np.full(T.size,np.nan,np.float32)
SH[np.in1d(T,T0)] = csv['specific_humidity'][np.in1d(T0,T)]
es = 0.6108 * np.exp((17.27*Ta)/(Ta+237.3)) * 1000    # [Pa]
ea = SH * (Pa*1000) / (0.378*SH + 0.622)    # [Pa]
ea[ea>es] = es[ea>es]
VPD = es - ea    # [Pa]
VPD /= 100

WS = np.full(T.size,np.nan,np.float32)
WS[np.in1d(T,T0)] = np.sqrt(csv['wind_u']**2+csv['wind_v']**2)[np.in1d(T0,T)]

Precip = np.full(T.size,np.nan,np.float32)
Precip[np.in1d(T,T0)] = csv['total_precipitation'][np.in1d(T0,T)]

Rg = np.full(T.size,np.nan,np.float32)
Rg[np.in1d(T,T0)] = csv['shortwave_radiation'][np.in1d(T0,T)]

LWin = np.full(T.size,np.nan,np.float32)
LWin[np.in1d(T,T0)] = csv['longwave_radiation'][np.in1d(T0,T)]

df = pd.DataFrame({'Ta_era':Ta, 'Pa_era':Pa, 'VPD_era':VPD, 'WS_era':WS, 'Precip_era':Precip, 'Rg_era':Rg, 'LWin_era':LWin, 'LWincalc_era':LWin})
df.index = T
df['Ta_era'] = df['Ta_era'].interpolate()
df['Pa_era'] = df['Pa_era'].interpolate()
df['VPD_era'] = df['VPD_era'].interpolate()
df['WS_era'] = df['WS_era'].interpolate()
df['Precip_era'] = df['Precip_era'].interpolate()
df['Rg_era'] = df['Rg_era'].interpolate()
df['LWin_era'] = df['LWin_era'].interpolate()
df['LWincalc_era'] = df['LWincalc_era'].interpolate()

for year in [2016,2017,2018,2019]:
    t = pd.date_range('%d-01-01 00:30'%year,'%d-12-31 23:30'%year,freq='h')
    msk = np.in1d(T,t)
    df[msk].to_csv('US-Ne2_%d.csv'%year,index=False)

In [ ]:
qcv_dir = output_dir + site_name + '/01_qc_visual/qcv_files/'
qcv_dir

In [ ]:
qcv_list = glob.glob(qcv_dir+ '*.csv')
qcv_list

In [ ]:
years = []
for item in qcv_list:
    file_name = os.path.basename(item)
    file_prefix = file_name.split('.')[0]
    year = file_prefix.split('_')[-1]
    years.append(year)

years.sort()

In [ ]:
years

In [ ]:
site_name

In [ ]:
tmp = df.iloc[100:200]
tmp

In [ ]:
a_row = df.iloc[111:222]
a_row['tair']

In [ ]:
tmp['VPD_era'] = tmp.apply (lambda row: create_vpd(row) , axis=1)
_tmp_data = tmp[selected_cols].rename(columns=new_col_names)
_tmp_data['LWincalc_era'] = _tmp_data['LWin_era']

_tmp_data = _tmp_data.interpolate()
_tmp_data